In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy import stats
from pprint import pprint
import sys
from colorama import Style, Fore
import warnings
import time 
import datetime as dt
import math
from glob import glob
import openpyxl



warnings.filterwarnings(action = 'ignore')

rc = {
    "axes.facecolor": "#F6F6F6",
    "figure.facecolor": "#F6F6F6",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7",
    "font.family": "malgun gothic",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4
}

sns.set(rc=rc)


red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
mgt = Style.BRIGHT + Fore.MAGENTA
gld = Style.BRIGHT + Fore.YELLOW
res = Style.RESET_ALL

In [ ]:

data = pd.read_csv("C:/doit/BigContest2023/2023빅콘테스트_어드밴스드리그_예술의전당.csv")

# 엑셀 파일 경로 지정
excel_file = "C:/doit/BigContest2023/seat_coordinates.xlsx"

# 엑셀 파일을 읽어와서 DataFrame으로 변환
seat_coordinates = pd.read_excel(excel_file)

file_path = 'C:/Users/honey/OneDrive/바탕 화면/예술의 전당 프로젝트/새 폴더/장르별_빈좌석포함데이터/'

full_path = 'C:/Users/honey/OneDrive/바탕 화면/예술의 전당 프로젝트/새 폴더/장르별_빈좌석포함데이터/*.csv'

pattern = '/*.csv'

glob(full_path)

file_list = glob(full_path)
dataframe_dict = {}
file_list[0].split('\\')[1].split('_')[0]

dataframe_dict = {}


for file in file_list: 
    a = file.split('\\')[1].split('_')[0]
    key = a
    dataframe_dict[key] = pd.read_csv(file)

In [ ]:
# 전처리
df_for_time= dataframe_dict['클래식']
df_for_time['선예매시작일'] = df_for_time['pre_open_date']
df_for_time['예매시작일'] = df_for_time['open_date']
df_for_time = df_for_time[['performance_code', '공연일', 'X','Y','Z', '전체거래시간','선예매시작일','예매시작일','seat']]



 # 거래시간 칼럼들 다 date_time 데이터타입으로 변환
df_for_time['전체거래시간'] = pd.to_datetime(df_for_time['전체거래시간'])
# 시간과 분을 0으로 추가
df_for_time['선예매시작일'] = pd.to_datetime(df_for_time['선예매시작일']).apply(lambda x: x.replace(hour=0, minute=0))
df_for_time['예매시작일'] = pd.to_datetime(df_for_time['예매시작일']).apply(lambda x: x.replace(hour=0, minute=0))

df_for_time.dropna(subset=['전체거래시간'], inplace=True)




result_time = pd.DataFrame(columns=['performance_code', '공연일','X','Y','Z', '전체거래시간','선예매시작일','예매시작일','예매걸린시간','첫예매순위','seat'])
unique_performance = df_for_time['performance_code'].unique().tolist()
count = 0 #162 끝


for specific_performance in unique_performance:
    filtered_df = df_for_time[df_for_time['performance_code'] == specific_performance]

    unique_date_number = filtered_df['공연일'].unique().tolist()
    
    

    for specific_date in unique_date_number:
        filtered_df2 = filtered_df[filtered_df['공연일'] == specific_date]

        # filtered_df2에서 전체예매시간이 가장빠른 행 찾기
        # 그 행을 기준으로 데이터프레임에서 시간빼기 시간으로 칼럼을 새로 만들기
        # 해당칼럼으로 히트맵

       



        # 첫예매걸린시간을 구하고 다른 칼럼들에 첫예매걸린시간과의 차를 구함
        fast = filtered_df2['전체거래시간'].min()
        filtered_df2['계산값'] = filtered_df2['전체거래시간'] - fast

        
        filtered_df2['첫예매순위'] = filtered_df2['계산값'].rank(method='min')

        result_time = pd.concat([result_time, filtered_df2], ignore_index=True)
        
        
        count += 1
        print(count)


result_time['계산값'] = result_time['계산값'] .dt.total_seconds() / 60



In [ ]:
# 전처리
df_for_time= dataframe_dict['클래식']
df_for_time['선예매시작일'] = df_for_time['pre_open_date']
df_for_time['예매시작일'] = df_for_time['open_date']
df_for_time = df_for_time[['전체공연시간', 'X','Y','Z', '전체거래시간','선예매시작일','예매시작일']]
df_for_time.dropna(subset=['전체거래시간'], inplace=True)


 # 거래시간 칼럼들 다 date_time 데이터타입으로 변환
df_for_time['전체거래시간'] = pd.to_datetime(df_for_time['전체거래시간'])
# 시간과 분을 0으로 추가
df_for_time['선예매시작일'] = pd.to_datetime(df_for_time['선예매시작일']) #.apply(lambda x: x.replace(hour=0, minute=0))
df_for_time['예매시작일'] = pd.to_datetime(df_for_time['예매시작일']) #.apply(lambda x: x.replace(hour=0, minute=0))



# 데이터프레임을 합칠 빈 데이터프레임 만들기
result_time = pd.DataFrame(columns=['전체공연시간','X','Y','Z', '전체거래시간','선예매시작일','예매시작일','예매걸린시간','예매시간종류'])

# 전체 공연시간을 이용하여 공연 특정하기
unique_performance = df_for_time['전체공연시간'].unique().tolist()




# 전체공연시간에 따라 데이터프레임을 차례로 나누기
for specific_performance in tqdm(unique_performance):
    filtered_df = df_for_time[df_for_time['전체공연시간'] == specific_performance]




#찾아서 넣는것보다 그냥 리스트로 만들어서 한번에 합치는게 훨씬 빠름
    time_list = pd.DataFrame(columns=['예매걸린시간'])
    type_list = pd.DataFrame(columns=['예매시간종류'])


    #선예매시작일에 값이 존재하면
    if pd.notna(filtered_df['선예매시작일'].iloc[0]):
        
        #그 전체거래시간 데이터프레임의 인덱스와 행을 차례로 배출
        for i, row in enumerate(filtered_df['전체거래시간']):
            #print(idx, row)
            
            #만약 전체거래시간이 예매시작일보다 늦으면
            if row > filtered_df['예매시작일'].iloc[0]:
            

                
                time_list.loc[i] = row - filtered_df['예매시작일'].iloc[0]
                type_list.loc[i] = '일반예매'

            
            #만약 전체거래시간이 예매시작일보다 빠르면
            else:
                #전체거래시간에서 선예매시작일을 빼고 예매걸린시간에 넣고 선예매라고 명시
                time_list.loc[i] = row - filtered_df['선예매시작일'].iloc[0]
                type_list.loc[i] = '선예매'
            
    #선예매시작일에 값이 없으면
    else :
        for i, row in enumerate(filtered_df['전체거래시간']):
            #전체거래시간에서 예매시작일을 빼고 그것을 예매걸린시간에 넣고 일반예매라고 명시
            time_list.loc[i] = row - filtered_df['예매시작일'].iloc[0]
            type_list.loc[i] = '일반예매'

        # 순위도 옆에 넣기 순위는 무엇을 기준으로 하나 빠른 것 기준이기 때문에 모두 같을 것
        # 다만 pre와 open을 구분할 수는 있음 하지만 그만저도 open에서 제일 빠른것을 기준으로 반으로 나눈 것과 같음
        #filtered_df2['첫예매순위'] = filtered_df2['계산값'].rank(method='min')
        #filtered_df2['오픈순위'] = filtered_df2['예매걸린시간'].rank(method='min')

    filtered_df = filtered_df.reset_index(drop=True)
    #여기서는 정상적으로 합쳐짐
    concat_df = pd.concat([filtered_df, time_list, type_list], axis=1, ignore_index = False)

    result_time = pd.concat([result_time, concat_df], ignore_index=True)

    time_list = pd.DataFrame()
    type_list = pd.DataFrame()


#수정필요
#result_time['예매걸린시간'] = result_time['예매걸린시간'].apply(pd.to_datetime, format='%H:%M:%S').applymap(lambda x: x.strftime('%H:%M'))

